In [11]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense 
import matplotlib.pyplot as plt
import plotly.graph_objects as go 
import plotly.express as px 
import datetime
from datetime import timedelta
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [12]:
df = pd.read_csv('GOOG.csv')
df

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,GOOG,2016-06-14 00:00:00+00:00,718.27,722.470,713.1200,716.48,1306065,718.27,722.470,713.1200,716.48,1306065,0.0,1.0
1,GOOG,2016-06-15 00:00:00+00:00,718.92,722.980,717.3100,719.00,1214517,718.92,722.980,717.3100,719.00,1214517,0.0,1.0
2,GOOG,2016-06-16 00:00:00+00:00,710.36,716.650,703.2600,714.91,1982471,710.36,716.650,703.2600,714.91,1982471,0.0,1.0
3,GOOG,2016-06-17 00:00:00+00:00,691.72,708.820,688.4515,708.65,3402357,691.72,708.820,688.4515,708.65,3402357,0.0,1.0
4,GOOG,2016-06-20 00:00:00+00:00,693.71,702.480,693.4100,698.77,2082538,693.71,702.480,693.4100,698.77,2082538,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,GOOG,2021-06-07 00:00:00+00:00,2466.09,2468.000,2441.0725,2451.32,1192453,2466.09,2468.000,2441.0725,2451.32,1192453,0.0,1.0
1254,GOOG,2021-06-08 00:00:00+00:00,2482.85,2494.495,2468.2400,2479.90,1253253,2482.85,2494.495,2468.2400,2479.90,1253253,0.0,1.0
1255,GOOG,2021-06-09 00:00:00+00:00,2491.40,2505.000,2487.3300,2499.50,1006337,2491.40,2505.000,2487.3300,2499.50,1006337,0.0,1.0
1256,GOOG,2021-06-10 00:00:00+00:00,2521.60,2523.260,2494.0000,2494.01,1561733,2521.60,2523.260,2494.0000,2494.01,1561733,0.0,1.0


In [13]:
def check(df):
    l=[]
    columns = df.columns
    for col in columns:
        dtypes = df[col].dtypes
        nunique = df[col].nunique()
        sum_null = df[col].isnull().sum()
        l.append([col,dtypes,nunique,sum_null])
    df_check = pd.DataFrame(l)
    df_check.columns = ['column', 'dtypes','nunique','sum_null']
    return df_check
check(df)

,column,dtypes,nunique,sum_null
0,symbol,object,1,0
1,date,object,1258,0
2,close,float64,1247,0
3,high,float64,1240,0
4,low,float64,1244,0
5,open,float64,1227,0
6,volume,int64,1258,0
7,adjClose,float64,1247,0
8,adjHigh,float64,1240,0
9,adjLow,float64,1244,0


In [14]:
df.describe().T 

,count,mean,std,min,25%,50%,75%,max
close,1258.0,1.216317e+03,383.333358,668.260,9.608025e+02,1132.460,1.360595e+03,2521.60
high,1258.0,1.227431e+03,387.570872,672.300,9.687575e+02,1143.935,1.374345e+03,2526.99
low,1258.0,1.204176e+03,378.777094,663.284,9.521825e+02,1117.915,1.348557e+03,2498.29
open,1258.0,1.215261e+03,382.446995,671.000,9.590050e+02,1131.150,1.361075e+03,2524.92
volume,1258.0,1.601590e+06,696017.226844,346753.000,1.173522e+06,1412588.500,1.812156e+06,6207027.00
adjClose,1258.0,1.216317e+03,383.333358,668.260,9.608025e+02,1132.460,1.360595e+03,2521.60
adjHigh,1258.0,1.227431e+03,387.570873,672.300,9.687575e+02,1143.935,1.374345e+03,2526.99
adjLow,1258.0,1.204176e+03,378.777099,663.284,9.521825e+02,1117.915,1.348557e+03,2498.29
adjOpen,1258.0,1.215261e+03,382.446995,671.000,9.590050e+02,1131.150,1.361075e+03,2524.92
adjVolume,1258.0,1.601590e+06,696017.226844,346753.000,1.173522e+06,1412588.500,1.812156e+06,6207027.00


In [31]:
import plotly.io as pio 
pio.renderers.default = 'vscode'
# pio.renderers.default = 'png'
# pio.renderers.default = 'browser'
fig = px.line(df, x='date', y='close', title='Close Prices vs. Date')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Closing Price')
fig.show()

In [32]:
fig = px.line(df, x='date', y='high', title='Highest price vs Date')
fig.update_xaxes(title="Date")
fig.update_yaxes(title='Hight price')
fig.show()

In [33]:
fig = px.line(df, x='date', y='low', title='Low prive vs Date')
fig.update_xaxes(title='date')
fig.update_yaxes(title='Low price')
fig.show()

In [34]:
fig = px.line(df, x='date', y='open', title='Open Prices vs. Date')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Closing Price')

fig.show()

In [35]:
fig = px.line(df, x='date', y='volume', title='Volume Prices vs. Date')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Volume')

fig.show()

In [36]:
fig = px.line(df, x='date', y='adjClose', title='Adj Close Prices vs. Date')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='adjClose')

fig.show()


In [37]:
fig = px.line(df, x='date', y='adjHigh', title='Adj High Prices vs. Date')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='adjHigh')

fig.show()

In [38]:
fig = px.line(df, x='date', y='adjLow', title='Adj Low Prices vs. Date')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='adjLow')

fig.show()

In [39]:
fig = px.line(df, x='date', y='adjOpen', title='Adj Open Prices vs. Date')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='adjOpen')

fig.show()

In [40]:
fig = px.line(df, x='date', y='adjVolume', title='Adj Volume Prices vs. Date')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='adjVolume')

fig.show()

In [41]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

In [42]:
stock = df[['date', 'close', 'high', 'low', 'open', 'volume']]

In [44]:
scaler = MinMaxScaler()
normalData = stock[['open', 'high', 'low', 'volume', 'close']].copy()
normalData = scaler.fit_transform(normalData)



In [46]:
trainData, testData = train_test_split(normalData, test_size=0.2, shuffle=False)

In [55]:
train_df = pd.DataFrame(trainData,columns=['open','high','low','volume','close'])
test_df = pd.DataFrame(testData,columns=['open','high','low','volume','close'])

In [58]:
def generateSequence(df,seqLength=50):
    x = df[['open','high','low','volume','close']].reset_index(drop=True)
    y = df[['open','high','low','volume','close']].reset_index(drop=True)

    sequences = []
    labels = []

    for index in range(len(x)-seqLength+1):
        sequences.append(x.iloc[index: index+seqLength].values)

    sequences = np.array(sequences)
    labels = np.array(labels)

    return sequences,labels

In [59]:
trainSequences, trainLabels = generateSequence(train_df)
testSequences, testLabels = generateSequence(test_df)

In [60]:
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(50, 5)),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50),
    Dropout(0.2),
    Dense(units=5)
])

In [61]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50, 50)         │        11,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,855 (202.56 KB)

 Trainable params: 51,855 (202.56 KB)

 Non-trainable params: 0 (0.00 B)

In [64]:
epochs = 200
batch_size = 32

history = model.fit(
    trainSequences,
    trainLabels,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(testSequences, testLabels),
    verbose=1
)

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 957
'y' sizes: 0
